In [ ]:
#---nightvision based on time and drowsiness detection with eye and sanding message and email ---

import cv2
import dlib
from imutils import face_utils
from scipy.spatial import distance
from pygame import mixer
import imutils
from twilio.rest import Client
import smtplib
import datetime
from email.message import EmailMessage

def sms(body1, from1, to1):
    account_ssid='AC9eb13e8523a6c0d87e6e8b8248ea1c38'
    auth_token='4dad115dd4fd0b57d306f22e519e5dd0'
    client=Client(account_ssid,auth_token)
    message=client.messages\
    .create(
        body=body1,
        from_=from1,
        to=to1
    )

def email_alert(subject, body, to):
    msg=EmailMessage()
    msg.set_content(body)
    
    msg['subject']=subject
    msg['to']=to
    
    user="wave01072024@gmail.com"
    msg['from']=user
    password="tgdjbqsityxwzuzs"
    
    server=smtplib.SMTP("smtp.gmail.com",587)
    server.starttls()
    server.login(user,password)
    server.send_message(msg)
    
    server.quit()

mixer.init()
mixer.music.load("music.wav")

def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

thresh = 0.25
frame_check = 20

detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS["right_eye"]

cap = cv2.VideoCapture(0)
flag = 0
numk = 0

# Function to apply night vision effect
def apply_night_vision(frame):
    # Split the frame into its color channels
    b, g, r = cv2.split(frame)

    # Enhance the green channel
    g = cv2.equalizeHist(g)

    # Merge back the channels
    night_vision_frame = cv2.merge((b, g, r))

    return night_vision_frame

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Check current time
    current_time = datetime.datetime.now().time()
    if current_time < datetime.time(6, 0) or current_time > datetime.time(19, 0):
        # Apply night vision effect to the frame
        frame = apply_night_vision(frame)
    else:
        # Use normal camera settings
        pass
    
    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    subjects = detect(gray, 0)
    
    for subject in subjects:
        shape = predict(gray, subject)
        shape = face_utils.shape_to_np(shape)
        
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        ear = (leftEAR + rightEAR) / 2.0
        
        eyeHull = cv2.convexHull(leftEye)
        cv2.drawContours(frame, [eyeHull], -1, (0, 255, 0), 1)
        eyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [eyeHull], -1, (0, 255, 0), 1)
        
        if ear < thresh:
            flag += 1
            if flag >= frame_check:
                numk = numk + 1
                cv2.putText(frame, "DROWSINESS ALERT!", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                if numk == 11:
                    email_alert("hey", "drowsiness has been detected from mohan, please contact the driver", "suppuratnakar@gmail.com")
                    sms('Suspicious activities are detected please contact the driver', '+12675926853', '+919731247737')
                if not mixer.music.get_busy():
                    mixer.music.play()
        else:
            flag = 0
            if mixer.music.get_busy():
                mixer.music.stop()
    
    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()